In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime

## 1. Extract data from files

In [ ]:

df = pd.DataFrame()

files = [file for file in os.listdir('./SalesAnalysis/Sales_Data/')]

for file in files:
    
    data = pd.read_csv('./SalesAnalysis/Sales_Data/'+file)
    df = pd.concat([df, data])


df.to_csv('./SalesAnalysis/all_data.csv',index=False)
df.head()



In [ ]:
df.shape

## 2. Data cleansing 

In [ ]:
## Drop nan rows
nan_df = df[df.isna().any(axis=1)]

##drop all nan 
df = df.dropna(how='all')


In [ ]:
## drop duplicated header rows - could be done when importing the data
df = df[df['Order Date'].str[0:2] != 'Or']

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'])
df['Price Each'] = pd.to_numeric(df['Price Each'])
df.head()

In [ ]:
## Business Question 1: What is the best month for sales and total sales in that month?

In [ ]:
## add hour, month and total sales columns
df['Sales'] = df['Quantity Ordered'] * df['Price Each']
df['Month'] = pd.DatetimeIndex(df['Order Date']).month
df['Hours'] = pd.DatetimeIndex(df['Order Date']).hour


In [ ]:
## Apply function to extract data

def get_city(address):
    return address.split(',')[1] 

def get_state(address):
    return address.split(',')[2].split(' ')[1]


df['City'] = df['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})")

In [ ]:
df.head()

In [ ]:
%matplotlib inline

In [ ]:
df_Months = df[['Month','Sales']]
results = df_Months.groupby('Month').sum()
months = range(1,13)

In [ ]:
plt.bar(months, results['Sales'])
plt.xticks(months)
plt.ylabel('Sales in USD')
plt.xlabel('Months')

## Business Question 2: What city has the highest number of sales

In [ ]:
df_Cities = df[['City','Sales']]

## to aviod inconsistancies
cities = [city for city , df in df_Cities.groupby('City')]
results = df_Cities.groupby('City').sum()

plt.bar(cities, results['Sales'])
plt.xticks(cities,rotation='vertical')
plt.ylabel('Sales')
plt.xlabel('Cities')

In [ ]:
df_Cities_Months = df[['Month','City','Sales']]

df_Cities_Months.groupby(['Month','City']).sum().unstack().plot(kind='line',stacked = False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Business Question 3: What time should we display advertisements to maximise conversion rate?

In [ ]:
df_month_group_hours = df[['City','Hours','Quantity Ordered']]
df_month_group_hours = df_month_group_hours.groupby('Hours').count()
hours = [hour for hour, df in df_month_group_hours.groupby('Hours')]

In [ ]:
plt.plot(hours, df_month_group_hours['Quantity Ordered'])
plt.xticks(hours)
plt.ylabel('Quantit')
plt.xlabel('Hours')


In [ ]:
df_month_group_hours = df[['City','Hours','Quantity Ordered']]
df_month_group_hours = df_month_group_hours.groupby(['Hours','City']).sum().unstack().plot(kind='line',stacked = False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()



## Business Question 4: What products were sold together?

In [ ]:
df_duplicate_OI = df[df['Order ID'].duplicated(keep=False)]
df_duplicate_OI['Grouped_Prod'] = df_duplicate_OI.groupby('Order ID')['Product'].transform(lambda x: ',' .join(x))

df_duplicate_OI = df_duplicate_OI[['Order ID','Grouped_Prod']].drop_duplicates()
#df_duplicate_OI

In [ ]:
from itertools import combinations
from collections import Counter

In [ ]:
## to get 2 items sold together

count = Counter()

for row in df_duplicate_OI['Grouped_Prod']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

print(count.most_common(10))

## Business Question 5: What product was sold most


In [ ]:
df_product_Grouped_All = df.groupby('Product')
quantity_ordered = df_product_Grouped_All.sum()['Quantity Ordered']
df_prices = df.groupby('Product').mean()['Price Each']

products = [product for product, df in df_product_Grouped_All]

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(products, quantity_ordered)
ax2.plot(products, df_prices, color='r')


ax1.set_xlabel('Products')
ax1.set_ylabel('Quantity')
ax2.set_ylabel('Prices (Avg)', color = 'r')
ax1.set_xticklabels(products, fontsize=8, rotation = 'vertical')
plt.show()



In [ ]:
df_Product_month_group = df[['Product','Month','Quantity Ordered']]
df_Product_month_group = df_Product_month_group.groupby(['Month','Product']).sum()
df_Product_month_group.reset_index


In [ ]:
df_Product_month_group.unstack().plot(kind='line',stacked = False)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


In [ ]:
# Top 5 product sold
df_Product_grp_Qty = df[['Product','Quantity Ordered']]
df_Product_grp_Qty = df_Product_grp_Qty.groupby(['Product']).sum()
df_Product_grp_Qty.reset_index

df_Product_grp_Qty['Ranking'] = df_Product_grp_Qty['Quantity Ordered'].rank(method='dense',ascending=False)
df_Product_grp_Qty = df_Product_grp_Qty[df_Product_grp_Qty['Ranking'] <=5].sort_values(by=['Ranking']) 

df_Product_grp_Sales = df[['Product','Sales']]
df_Product_grp_Sales = df_Product_grp_Sales.groupby(['Product']).sum()

pd.merge(df_Product_grp_Qty, df_Product_grp_Sales, on='Product', how= 'left' )
